In [ ]:
#importing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import smart_open

import logging
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer



from gensim.models import Word2Vec
from sklearn.manifold import TSNE



In [ ]:
!pip install transformers
from transformers import BertTokenizer


from tqdm.notebook import tqdm


from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
labels_dict = {'Artificial Intelligence Applied Researcher': 0,
 'Artificial Intelligence/Machine Learning Engineer': 1,
 'Applications Support Engineer': 2,
 'Associate Applications Support Engineer': 3,
 'Associate Business Analyst': 4,
 'Associate Data Centre Operations Engineer': 5,
 'Associate Database Support Engineer': 6,
 'Associate Embedded Systems Engineer': 7,
 'Associate Infrastructure Engineer': 8,
 'Associate Infrastructure Support Engineer': 9,
 'Associate Network Engineer': 10,
 'Associate Operations Centre Support Engineer': 11,
 'Associate Radio Frequency Engineer': 12,
 'Associate Security Analyst': 13,
 'Associate Software Engineer': 14,
 'Systems Support Engineer': 15,
 'Associate Systems Support Engineer': 16,
 'Associate UI Designer': 17,
 'Associate UX Designer': 18,
 'Automation and Orchestration Engineer': 19,
 'Business Intelligence Director': 20,
 'Business Intelligence Manager': 21,
 'Business Analyst/Artificial Intelligence Translator': 22,
 'Business Architect': 23,
 'Business Development Manager': 24,
 'Chief Data Officer/Chief Artificial Intelligence Officer': 25,
 'Channel Sales Manager': 26,
 'Chief Information Officer': 27,
 'Chief Technology Officer': 28,
 'Chief Information Security Officer': 29,
 'Customer Success Director': 30,
 'Customer Success Manager': 31,
 'Cyber Risk Analyst': 32,
 'Cyber Risk Manager': 33,
 'Data Scientist/Artificial Intelligence Scientist': 34,
 'Data Analyst/Associate Data Engineer': 35,
 'Data Architect': 36,
 'Data Centre Operations Engineer': 37,
 'Data Engineer': 38,
 'Data Protection Executive': 39,
 'Data Protection Officer': 40,
 'Database Support Engineer': 41,
 'DevOps Engineer': 42,
 'Embedded Systems Engineer': 43,
 'Embedded Systems Engineering Manager': 44,
 'Enterprise Architect': 45,
 'Forensics Investigation Manager': 46,
 'Forensics Investigator': 47,
 'Group Data Protection Officer': 48,
 'Head of Data Science and Artificial Intelligence': 49,
 'Head of Infrastructure': 50,
 'Head of IT Audit': 51,
 'Head of Operations and Support': 52,
 'Head of Product': 53,
 'Head of Quality': 54,
 'Head of Sales': 55,
 'Head of Software Engineering': 56,
 'Incident Investigation Manager': 57,
 'Incident Investigator': 58,
 'Infrastructure Architect': 59,
 'Infrastructure Engineering Manager': 60,
 'Infrastructure Support Engineer': 61,
 'IT Audit Manager': 62,
 'IT Auditor': 63,
 'Lead UI Designer': 64,
 'Lead UX Designer': 65,
 'Marketing Director': 66,
 'Marketing Executive': 67,
 'Marketing Manager': 68,
 'Network Engineer': 69,
 'Operations Centre Support Engineer': 70,
 'Pre-Sales Consultant': 71,
 'Pre-Sales Director': 72,
 'Product Manager': 73,
 'Program Director': 74,
 'Program Manager': 75,
 'Project Manager/Scrum Master': 76,
 'Quality Assurance Engineer': 77,
 'Quality Assurance Manager': 78,
 'Quality Engineer': 79,
 'Quality Engineering Manager': 80,
 'Radio Frequency Engineer': 81,
 'Sales Account Manager': 82,
 'Sales Director': 83,
 'Sales Executive': 84,
 'Security Architect': 85,
 'Security Operations Analyst': 86,
 'Security Operations Manager': 87,
 'Senior Artificial Intelligence/Machine Learning Engineer': 88,
 'Senior Data Engineer': 89,
 'Senior Product Manager': 90,
 'Senior Security Engineer/Security Engineer': 91,
 'Software Architect': 92,
 'Software Engineer': 93,
 'Software Engineering Manager': 94,
 'Solutions Architect': 95,
 'SysOps Engineer': 96,
 'Threat Analysis Manager': 97,
 'UI Designer': 98,
 'UX Designer': 99,
 'Vulnerability Assessment and Penetration Testing Analyst': 100,
 'Vulnerability Assessment and Penetration Testing Manager': 101,
 'Infrastructure Engineer': 102,
 'Operations and Support Manager': 103}

In [ ]:
import pandas as pd
import io
df = pd.read_csv(io.StringIO(uploaded['sf_model_IT_Bert.csv'].decode('utf-8')))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'


In [ ]:
import tensorflow as tf
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Skill_description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Skill_description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(labels_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    labels_dict_inverse = {v: k for k, v in labels_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {labels_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:


seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
files.upload()
import helper



In [ ]:
import random

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        

    torch.save(model.state_dict(), f'checkpoint{epoch}.pth')
    files.download(f'checkpoint{epoch}.pth')    
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(labels_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('checkpoint1.pth', map_location=torch.device('cpu')))
# state_dict = torch.load('checkpoint.pth')
# model.load_state_dict(state_dict)

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

In [ ]:
np.argmax(predictions[2])

In [ ]:
y_val